In [1]:
from bs4 import BeautifulSoup
import requests
import urllib
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
import pandas as pd

### 1. Парсинг одной страницы

In [8]:
browser = webdriver.Chrome(ChromeDriverManager().install())
qry = 'мёд'
url = 'https://www.avito.ru/tomsk?cd=1&p=2&q='

browser.get(url + urllib.parse.quote_plus(qry))

page = browser.page_source

bs_page = BeautifulSoup(page)

C:\Users\roman\AppData\Local\Temp\ipykernel_1236\4135749229.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


In [9]:
bs_items = bs_page.find_all('div', attrs={'class': 'iva-item-body-KLUuy'})#Главная ссылка

bs_item = bs_items[0]

bs_title = bs_item.find('div', attrs={'class': 'iva-item-titleStep-pdebR'})
bs_title_a = bs_title.find('a')
title = bs_title_a.attrs['title']

bs_price_a = bs_item.find('div', attrs = {'class':'iva-item-priceStep-uq2CQ'})
bs_price_c = bs_price_a.find('meta', attrs = {'itemprop': "price"})
price = bs_price_c.attrs['content']

bs_geo = bs_item.find('div', attrs = {'class':'geo-root-zPwRk'})
bs_geo_a = bs_geo.find('div', attrs = {'class':'geo-georeferences-SEtee text-text-LurtD text-size-s-BxGpL'})
if bs_geo_a is not None:
    bs_geo_a = bs_geo_a.text

bs_time = bs_item.find('div', attrs = {'class':'iva-item-dateInfoStep-_acjp'})
bs_time_a = bs_time.find('span', attrs = {'class' :'tooltip-target-wrapper-mu94t'})
time = bs_time_a.text

bs_info = bs_item.find('div', attrs = {'class': 'iva-item-descriptionStep-C0ty1'})
bs_info_a = bs_info.find('div', attrs = {'class': 'iva-item-text-Ge6dR'})
info = bs_info_a.text


print(f'Наименование: {title}'+ ';' , f'Цена: {price}'+';', f'Локация: {bs_geo_a}'+';',f'Время пребывания: {time}'+';',f'Инфо: {info}',  sep = ' ')

Наименование: Мед Алтайский оптом в Томске; Цена: 160; Локация: р-н Ленинский; Время пребывания: 3 недели назад; Инфо: Продам Алтайский мёд урожая 2022 года оптом. Район пасеки город Змеиногорск. По составу мед подсолнечник-гречиха. 
Цена 170 рублей за килограмм с доставкой. Мед в куботейнерах по 33 кг.


In [10]:
items = []

for bs_item in bs_items:
    bs_title = bs_item.find('div', attrs={'class': 'iva-item-titleStep-pdebR'})
    bs_title_a = bs_title.find('a')
    title = bs_title_a.attrs['title']
        
    bs_price_a = bs_item.find('div', attrs = {'class':'iva-item-priceStep-uq2CQ'})
    bs_price_c = bs_price_a.find('meta', attrs = {'itemprop': "price"})
    price = bs_price_c.attrs['content']  
                          
    bs_geo = bs_item.find('div', attrs = {'class':'geo-root-zPwRk'})
    bs_geo_a = bs_geo.find('div', attrs = {'class':'geo-georeferences-SEtee text-text-LurtD text-size-s-BxGpL'})
    if bs_geo_a is not None:
        geo = bs_geo_a.text
           
    
    bs_time = bs_item.find('div', attrs = {'class':'iva-item-dateInfoStep-_acjp'})
    bs_time_a = bs_time.find('span', attrs = {'class' :'tooltip-target-wrapper-mu94t'}) 
    time = bs_time_a.text
  
    
    bs_info = bs_item.find('div', attrs = {'class': 'iva-item-descriptionStep-C0ty1'})
    if bs_info is not None:
        bs_info_a = bs_info.find('div', attrs = {'class': 'iva-item-text-Ge6dR'})
        info = bs_info_a.text

        items.append({'title': title, 'price': price, 'lok': geo, 'time': time, 'info': info})

In [11]:
data = pd.DataFrame(items)
data

,title,price,lok,time,info
0,Мед Алтайский оптом в Томске,160,р-н Ленинский,3 недели назад,Продам Алтайский мёд урожая 2022 года оптом. Р...
1,"Алтайский Мед, перга, прополис Тмс в Томске",1500,р-н Ленинский,21 час назад,Мёд нaтурaльный бeз добaвoк из Экологически чи...
2,Мёд натуральный опт в Томске,170,р-н Ленинский,1 день назад,"Мед Алтайский оптом напрямую от производителя,..."
3,Milk and Honey Молоко и Мёд Рупи Каур в Томске,555,р-н Кировский,1 день назад,Milk and Honey | Молоко и Мёд | Рупи Каур.\nО ...
4,"Черный Лукич / Коллекция / Янка, Егор Летов в ...",0,р-н Кировский,1 день назад,"Продам коллекцию дисков Лукича, часть в заводс..."
...,...,...,...,...,...
88,Мёд в Новосибирске,200,р-н Октябрьский,2 дня назад,"Пчелиный мед, горный Алтай, сбор 2022го года, ..."
89,Конверт на выписку демисезонный в Красноярске,300,р-н Октябрьский,1 неделю назад,"Конверт на выписку демисезонный, 57 на 85, цве..."
90,Мёд в Кемерово,1000,р-н Октябрьский,2 недели назад,Качка 2022. Мед разнотравие.3л.1000 доставка. ...
91,Бортевой мед в Новосибирске,1500,р-н Октябрьский,3 дня назад,Дикий-бортевой мёд с Бурзянского района (Башки...


In [12]:
data.to_csv('data_1стр_13.04.2023.csv', index = False)

### 2. Парсинг нескольких страниц

In [6]:
browser = webdriver.Chrome(ChromeDriverManager().install())
url = ['https://www.avito.ru/tomsk?cd=1&p=1&q=',
     'https://www.avito.ru/tomsk?cd=1&p=2&q=',
     'https://www.avito.ru/tomsk?cd=1&p=3&q=',
      'https://www.avito.ru/tomsk?cd=1&p=4&q=',
      'https://www.avito.ru/tomsk?cd=1&p=5&q=']


def pr(url):
    qry = 'мёд'
    browser.get(url + urllib.parse.quote_plus(qry))
    return browser.page_source

items = []

b = [i for i in map(pr, url)]

for i in b:
    
    page = i
    bs_page = BeautifulSoup(page)
    bs_items = bs_page.find_all('div', attrs={'class': 'iva-item-body-KLUuy'})#Главная ссылка
    
    for bs_item in bs_items:
        bs_title = bs_item.find('div', attrs={'class': 'iva-item-titleStep-pdebR'})
        bs_title_a = bs_title.find('a')
        title = bs_title_a.attrs['title']

        bs_price_a = bs_item.find('div', attrs = {'class':'iva-item-priceStep-uq2CQ'})
        bs_price_c = bs_price_a.find('meta', attrs = {'itemprop': "price"})
        price = bs_price_c.attrs['content']  

        bs_geo = bs_item.find('div', attrs = {'class':'geo-root-zPwRk'})
        if bs_geo is not None:
            bs_geo_a = bs_geo.find('div', attrs = {'class':'geo-georeferences-SEtee text-text-LurtD text-size-s-BxGpL'})
            if bs_geo_a is not None:
                geo = bs_geo_a.text


        bs_time = bs_item.find('div', attrs = {'class':'iva-item-dateInfoStep-_acjp'})
        bs_time_a = bs_time.find('span', attrs = {'class' :'tooltip-target-wrapper-mu94t'}) 
        time = bs_time_a.text


        bs_info = bs_item.find('div', attrs = {'class': 'iva-item-descriptionStep-C0ty1'})
        if bs_info is not None:
            bs_info_a = bs_info.find('div', attrs = {'class': 'iva-item-text-Ge6dR'})
            info = bs_info_a.text

        items.append({'title': title, 'price': price, 'lok': geo, 'time': time, 'info': info})
            
data = pd.DataFrame(items)
data

C:\Users\roman\AppData\Local\Temp\ipykernel_1236\724614406.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


,title,price,lok,time,info
0,"Мёд дягилевый, дягилевый мед в Томске",1000,р-н Ленинский,1 неделю назад,"Цена за 3 литра.\nМёд 2022, вкусный, очень аро..."
1,Мёд в Томске,1250,р-н Кировский,16 марта 17:50,"Мёд.\nИз Кемеровской области, п/ст Арлюк.\n3 л..."
2,Мёд Качественный Алтaйcкий в Томске,1040,р-н Октябрьский,2 дня назад,"Доброгo Baм дня от пчеловода, предлагаем Алтaй..."
3,Аромамасло для свечей в Томске,384,р-н Ленинский,3 часа назад,Аромамасло для свечей «Роза и мёд». Производит...
4,Мёд алтайский и продукты пчеловодства в Томске,266,р-н Ленинский,3 недели назад,Алтайский мёд! Подсолнечник-гречиха Район пасе...
...,...,...,...,...,...
289,Продам книгу Рупи Каур «Молоко и мёд» белые ст...,300,р-н Ленинский,2 недели назад,Книге меньше недели.
290,Мёд в Новокузнецке,270,р-н Ленинский,3 недели назад,Всем здравствуйте.\nМега-Мега-Акция-Акция Акци...
291,Идеальный Мёд. Белокуриха в Новосибирске,600,р-н Ленинский,2 недели назад,"Продаю идеальный, очень вкусный правильный мёд..."
292,Бочонки под мёд в Новосибирске,150,р-н Ленинский,2 дня назад,Фото 1 туески берестяные 4 шт в наличии и бочо...


In [7]:
data.to_csv('data_13.04.2023.csv', index = False )